In [40]:
import os
import cv2
import numpy as np
import mediapipe as mp
import time
import math

In [41]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

mp_draw = mp.solutions.drawing_utils


In [42]:
last_save_time = 0
last_clear_time = 0
last_color_change = 0
cooldown = 10
canvas = None
saved_canvas = r"C:\Users\Nirajan\Desktop\Internship_Projects\Virtual_Pen_Project\Canvas"
cap = cv2.VideoCapture(1)

In [43]:
Onscreen_color = (0,255,255)
color_palette = [
    ((10, 10), (60, 60), (255, 255, 255)),    
    ((70, 10), (120, 60), (0, 255, 0)), 
    ((130, 10), (180, 60), (0, 0, 255)),   
    ((190, 10), (240, 60), (255, 0, 0)),
    ((250, 10), (300, 60), (0, 255, 255)), 
    ((310, 10), (360, 60), (255, 0, 255)),   
    ((370, 10), (420, 60), (0, 165, 255))
]
color_name = {
    (0, 255, 255): "Yellow",
    (255, 0, 255): "Purple",
    (255, 0, 0): "Blue",
    (0, 255, 0): "Green",
    (0, 0, 255): "Red",
    (255, 255, 255): "White",
    (0, 0, 0): "Black"
}

In [44]:
def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def thumbs_up(landmarks):
    thumb_tip = landmarks[4]
    thumb_ip = landmarks[3]
    folded_fingers = all(landmarks[i].y > landmarks[i - 2].y for i in [8, 12, 16, 20])
    return thumb_tip.y < thumb_ip.y and folded_fingers

def ok_sign(landmarks):
    thumb = [landmarks[4].x, landmarks[4].y]
    index = [landmarks[8].x, landmarks[8].y]
    d = distance(thumb, index)
    return d < 0.05

In [45]:
prev_left = (0,0)
prev_right = (0,0) #Initializing points for drawing


canvas_cleared = False
canvas_saved = False

while True: 
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame ,1 )

    h, w, _ = frame.shape

    if canvas is None:
        canvas = np.zeros_like(frame) #Creates cnavas of same size as the frame


    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    current_time = time.time()
    
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            label = handedness.classification[0].label  #Classifies if the hand is left or right
            landmarks = hand_landmarks.landmark

            if label == "Right":
                # Draw with index finger (landmark 8)
                ix = int(landmarks[8].x * w)
                iy = int(landmarks[8].y * h)

                for idx, (start, end, color) in enumerate(color_palette):
                    if start[0] <= ix <= end[0] and start[1] <= iy <= end[1]:
                        if selected_color_index != idx and (current_time - last_color_change > cooldown):  # Prevent redundant changes
                            Onscreen_color = color
                            selected_color_index = idx
                            last_color_change = current_time
                            color_name = color_name_map.get(Onscreen_color, str(Onscreen_color))
                            print(f"Color changed to {color_name}")
                        break
                else:
                    selected_color_index = -1


                if prev_right == (0, 0):
                    prev_right = (ix, iy)

                cv2.line(canvas, prev_right, (ix, iy), Onscreen_color , 5)
                prev_right = (ix, iy)

                if thumbs_up(landmarks):
                    if not canvas_saved and current_time - last_save_time > cooldown:
                        filename = os.path.join(saved_canvas, f"drawing_{int(current_time)}.png")
                        cv2.imwrite(filename, canvas)
                        print(f"Canvas saved as '{filename}'")
                        last_save_time = current_time
                else:
                    canvas_saved = False
        

            elif label == "Left":
                # Erase with middle finger (landmark 12)
                x = int(landmarks[12].x * w)
                y = int(landmarks[12].y * h)

                if prev_left == (0, 0):
                    prev_left = (x, y)

                cv2.line(canvas, prev_left, (x, y), (0, 0, 0), 50)  # Thick black line = eraser
                prev_left = (x, y)

                if ok_sign(landmarks):
                    if not canvas_cleared and current_time - last_clear_time > cooldown:
                        canvas = np.zeros_like(frame)
                        print("Canvas cleared!")
                        last_clear_time = current_time
                else:
                    canvas_cleared = False

            # Draw landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    else:
        # Reset points if hands are not detected
        prev_right = (0, 0)
        prev_left = (0, 0)


    # Merge canvas and frame
    frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    for (start, end, color) in color_palette:
        cv2.rectangle(frame, start, end, color, -1)
        cv2.rectangle(frame, start, end, (0, 0, 0), 2)

    cv2.putText(frame, f"Selected: {color_name}", (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)


    cv2.imshow("Virtual Pen", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

Canvas saved as 'C:\Users\Nirajan\Desktop\Internship_Projects\Virtual_Pen_Project\Canvas\drawing_1747830019.png'
Canvas cleared!
Color changed to Red
Canvas saved as 'C:\Users\Nirajan\Desktop\Internship_Projects\Virtual_Pen_Project\Canvas\drawing_1747830029.png'
Canvas cleared!
Canvas saved as 'C:\Users\Nirajan\Desktop\Internship_Projects\Virtual_Pen_Project\Canvas\drawing_1747830040.png'
Canvas cleared!
